# Crowdedness Data

## Imports <a name="imports"></a>

In [55]:
import pandas as pd
from datetime import datetime
import numpy as np

from sklearn.preprocessing import LabelEncoder

## Import Data <a name="importData"></a>

### Crowdedness Data <a name="crowdData"></a>
The crowdedness count per camera, per day, per hour

In [98]:
#Load in the data
crowd_df = pd.read_excel("../../../Data_thesis/CMSA/cmsa_data.xlsx")

In [99]:
#Example Contents
crowd_df.head()

,richting,Timestamp,StartPoint,StartPointName,EndPoint,EndPointName,MinMeasuredTime,MaxMeasuredTime,AvgMeasuredTime,MedianMeasuredTime,...,tijdstip,weekdag,tijdstip_verschoven,verschoven,datum_verschoven,weekdag_verschoven,tijdstip(15_min),weeknr,weekdag_tijd,weekend filter
0,2,2018-03-11 00:05:00,1,Start,2,End,0,0,0,0,...,00:04:00,NaN,1900-01-01 00:04:00,1,2018-03-10,NaN,1900-01-01 00:00:00,2018-10,1900-01-07,weekend
1,02R,2018-03-11 00:05:00,1,Start,2,End,0,0,0,0,...,00:04:00,NaN,1900-01-01 00:04:00,1,2018-03-10,NaN,1900-01-01 00:00:00,2018-10,1900-01-07,weekend
2,Oude Kennissteeg Occ wifi,2018-03-11 00:05:00,1,Start,2,End,0,0,0,0,...,00:04:00,NaN,1900-01-01 00:04:00,1,2018-03-10,NaN,1900-01-01 00:00:00,2018-10,1900-01-07,weekend
3,2,2018-03-11 00:10:00,1,Start,2,End,0,0,0,0,...,00:10:00,NaN,1900-01-01 00:10:00,1,2018-03-10,NaN,1900-01-01 00:00:00,2018-10,1900-01-07,weekend
4,02R,2018-03-11 00:10:00,1,Start,2,End,0,0,0,0,...,00:10:00,NaN,1900-01-01 00:10:00,1,2018-03-10,NaN,1900-01-01 00:00:00,2018-10,1900-01-07,weekend


### Crowdedness Data (Without Sensor column)
All the data resides in multiple files, where the filename equals the sensor number

In [100]:
blip_df = pd.read_csv("../../../Data_thesis/CMSA/BlipData.csv")

In [101]:
blip_df.head()

,Date,Sensor,CrowdednessCount,Hour
0,2018-04-01,GAWW-01,4271,0
1,2018-04-01,GAWW-01,3414,1
2,2018-04-01,GAWW-01,246,10
3,2019-01-01,GAWW-01,2788,3
4,2018-07-04,GAWW-01,1513,15


### Sensor Locations <a name="senData"></a>
The location of the cameras

In [102]:
#Load in the data
sensor_df = pd.read_csv("../../../Data_thesis/Open_Data/crowdedness_sensoren.csv", sep=";")

In [103]:
#Example contents
sensor_df.head()

,OBJECTNUMMER,Objectnummer,Soort,Voeding,Rotatie,Actief,Privacyverklaring,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Unnamed: 11
0,1,GAWW-03,Telcamera,Eigen voeding via lichtnet,115,Ja,NaN,"POINT(4.8973932,52.3725037)","POINT(52.3725037,4.8973932)","4,8973932","52,3725037",NaN
1,2,GAWW-03,WiFi sensor,Eigen voeding via lichtnet,115,Ja,https://www.amsterdam.nl/privacy/specifieke/pr...,"POINT(4.8973336,52.3725237)","POINT(52.3725237,4.8973336)","4,8973336","52,3725237",NaN
2,3,GAWW-02,WiFi sensor,Eigen voeding via lichtnet,115,Ja,https://www.amsterdam.nl/privacy/specifieke/pr...,"POINT(4.8988705,52.3737982)","POINT(52.3737982,4.8988705)","4,8988705","52,3737982",NaN
3,4,GAWW-02,Telcamera,Eigen voeding via lichtnet,115,Ja,NaN,"POINT(4.8989027,52.3737857)","POINT(52.3737857,4.8989027)","4,8989027","52,3737857",NaN
4,5,GAWW-01,WiFi sensor,Eigen voeding via lichtnet,0,Ja,https://www.amsterdam.nl/privacy/specifieke/pr...,"POINT(4.8997667,52.374627)","POINT(52.374627,4.8997667)","4,8997667","52,374627",NaN


## Functions

In [104]:
def SensorCoordinates(sensor_df, needed_sensors):
    """
    This function retrieves the Longitude and Latitude of the needed Sensors and returns these. 

    Input:
        - Sensor_df: DF with longitude and latitude of all the sensors in Amsterdam
        - needed_sensor (hyperparameter): List with all the sensors from which the location must be retrieved

    Output:
        - Dict file as follows --> SensorName = {"Longitude": <longitude>, "Latitude": <latitude>}
    """

    #Dict to saved the needed locations in
    locations_dict = {}

    #Select columns to use
    sensor_df = sensor_df[["Objectnummer", "LNG", "LAT"]]

    #Change Df into dict (Needed for operations on data)
    sensor_dict = sensor_df.to_dict("index")

    for k, v in sensor_dict.items():

        #Save only the cameras with the object nummer given above
        if v["Objectnummer"] in needed_sensors:

            #Replace the "." with "," to make sure the coordinates can be turned into floats
            v["LNG"] = float(v["LNG"].replace(",", "."))
            v["LAT"] = float(v["LAT"].replace(",", "."))

            #Save all contents in seperate dict
            locations_dict[v["Objectnummer"]] = {
                "Longitude": v["LNG"], "Latitude": v["LAT"]}

    return locations_dict

In [105]:
def CrowdednessData(crowd_df, blip_df, locations_dict, needed_sensors, gaww_02, gaww_03):
    """
    This function takes the crowdedness data from all the sensors within Amsterdam. The data from sensors that roughly measure the same place is aggregated under the 
    same sensor name and combined with the latitude and longitude of the sensor's measure area (see function SensorCoordinates). 

    Input:
        - crowd_df: Df with the crowdedness data of all the sensors
        - locations_dict; Dict with the longitude and latitude of the relevant sensor (constructed in function SensorCoordinates)
        - gaww_02/gaww_03: List with alternative sensor names 
    """

    crowd_df = pd.concat([crowd_df, blip_df],
                         sort=True).reset_index().drop(columns={"index"})

    #Group the counts of people per hour, per date, per camera
    crowd_df = crowd_df.groupby(["richting", "datum", "uur"])[
        "SampleCount"].sum().reset_index()
    
    #Rename the columns
    crowd_df = crowd_df.rename(index=str, columns={"richting": "Sensor", "datum": "Date", "uur": "Hour",
                                               "SampleCount": "CrowdednessCount"})
    
    #For the longitude number of the sensor
    crowd_df.insert(3, "SensorLongitude", 0)

    #For the latitude number of the sensor
    crowd_df.insert(4, "SensorLatitude", 0)

    #Change Df into dict
    crowd_dict = crowd_df.to_dict("index")

    #Loop over dict
    for k, v in crowd_dict.items():

        #Change camera name
        if v["Sensor"] in gaww_02:
            v["Sensor"] = "GAWW-02"

        #Change camera name
        elif v["Sensor"] in gaww_03:
            v["Sensor"] = "GAWW-03"

        if v["Sensor"] in needed_sensors:

            v["SensorLongitude"] = locations_dict[v["Sensor"]]["Longitude"]
            v["SensorLatitude"] = locations_dict[v["Sensor"]]["Latitude"]

        #Mulitply hour with 100 (Same structure as the other files)
        v["Hour"] *= 100

        if v["Hour"] == 0:
            v["Hour"] = 2400

    #Return from Dict
    full_df = pd.DataFrame.from_dict(crowd_dict, orient="index")

    #Onlt save the sensors for which the coordinates are known
    crowd_df = crowd_df[crowd_df["Sensor"].isin(needed_sensors)]

    #Group the multiple different sensor data from same date and hour together
    full_df = full_df.groupby(["Sensor", "Date", "Hour", "SensorLongitude",
                                 "SensorLatitude"])["CrowdednessCount"].sum().reset_index()

    crowd_df['SensorLongitude'] = LabelEncoder().fit_transform(crowd_df['SensorLongitude'])
    crowd_df['SensorLatitude'] = LabelEncoder().fit_transform(crowd_df['SensorLatitude'])

    return full_df

### Variables

In [106]:
#Sensors to use in Sensor Data
needed_sensors = ["GAWW-01", "GAWW-02", "GAWW-03", "GAWW-04", "GAWW-05", "GAWW-06", "GAWW-07", "GAWW-08", "GAWW-09",
                  "GAWW-10"]

#Alternative names Sensors
gaww_02 = [2, "02R", "2R", "Oude Kennissteeg Occ wifi"]
gaww_03 = [3, "03R"]

### Transforms Sensor DF

In [107]:
#Transform Sensor df
locations_dict = SensorCoordinates(sensor_df, needed_sensors)

### Construct Full DF

In [108]:
full_df = CrowdednessData(crowd_df, blip_df, locations_dict, needed_sensors, gaww_02, gaww_03)

In [109]:
#Example contents
full_df.head()

,Sensor,Date,Hour,SensorLongitude,SensorLatitude,CrowdednessCount
0,GAWW-02,2018-03-11,100.0,4.898903,52.373786,0.0
1,GAWW-02,2018-03-11,200.0,4.898903,52.373786,0.0
2,GAWW-02,2018-03-11,300.0,4.898903,52.373786,0.0
3,GAWW-02,2018-03-11,400.0,4.898903,52.373786,0.0
4,GAWW-02,2018-03-11,500.0,4.898903,52.373786,0.0


## DF to File <a name="CSV"></a>

In [110]:
full_df.to_csv("../../../Data_thesis/Full_Datasets/Crowdedness.csv", index=False)